In [10]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [18]:
test_day = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D10.parquet')
test_day.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
26961,2024-12-26,54.849998,55.400002,54.400002,55.009998,55.009998,13155800,VALE3.SA,0.160000,55.480276,NaN,55.009998,NaN,NaN,NaN,0.000000,0.16,38.934135,53.330002,60.189999,31.345124,-68.654876,-0.989594,-0.804075,-0.053182,2209347870
26962,2024-12-27,54.730000,55.080002,54.459999,54.740002,54.740002,16635300,VALE3.SA,-0.269997,55.332221,NaN,54.740002,NaN,NaN,NaN,0.269997,0.00,36.806222,53.330002,60.189999,29.186887,-70.813113,-1.031080,-0.849476,-0.049589,2192712570
26963,2024-12-30,54.900002,55.189999,54.549999,54.549999,54.549999,11250900,VALE3.SA,-0.190002,55.175777,NaN,54.549999,NaN,NaN,NaN,0.190002,0.00,35.242227,53.330002,60.189999,26.906355,-73.093645,-1.064313,-0.892443,-0.044532,2181461670
26964,2025-01-02,54.709999,55.099998,54.230000,54.250000,54.250000,17623900,VALE3.SA,-0.299999,54.990622,NaN,54.250000,NaN,NaN,NaN,0.299999,0.00,32.709994,53.330002,59.939999,25.122852,-74.877148,-1.092991,-0.932553,-0.041373,2163837770
26965,2025-01-03,53.900002,54.000000,52.880001,53.240002,53.240002,23608700,VALE3.SA,-1.009998,54.640498,NaN,53.240002,NaN,NaN,NaN,1.009998,0.00,25.572295,52.880001,59.590000,26.236911,-73.763089,-1.130935,-0.972229,-0.043294,2140229070


In [12]:
test_day['Ticker'].unique()

array(['ABEV3.SA', 'ITUB4.SA', 'MGLU3.SA', 'PETR4.SA', 'VALE3.SA'],
      dtype=object)

In [19]:
test_day_AMBEV = test_day[test_day['Ticker'] == 'ABEV3.SA'].reset_index(drop=True)
test_day_AMBEV['Prediction'] = np.NaN

test_day_AMBEV.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5730,2024-12-26,12.18,12.24,11.99,12.14,12.14,21166800,ABEV3.SA,-0.080000,12.675070,NaN,12.14,NaN,NaN,NaN,0.080000,0.0,25.409653,11.99,14.53,26.971287,-73.028713,0.028984,0.124149,-0.081700,-2455355247
5731,2024-12-27,12.16,12.19,11.97,12.07,12.07,37624000,ABEV3.SA,-0.070001,12.554056,NaN,12.07,NaN,NaN,NaN,0.070001,0.0,24.226632,11.97,14.53,22.814694,-77.185306,-0.017453,0.095829,-0.081110,-2492979247
5732,2024-12-30,12.06,12.16,11.74,11.74,11.74,38907400,ABEV3.SA,-0.330000,12.391245,NaN,11.74,NaN,NaN,NaN,0.330000,0.0,19.331009,11.74,14.50,23.595839,-76.404161,-0.066624,0.063338,-0.085231,-2531886647
5733,2025-01-02,11.72,11.89,11.49,11.60,11.60,33417900,ABEV3.SA,-0.139999,12.232996,NaN,11.60,NaN,NaN,NaN,0.139999,0.0,17.590948,11.49,14.50,24.684261,-75.315739,-0.117013,0.027268,-0.087404,-2565304547
5734,2025-01-03,11.49,11.55,11.27,11.33,11.33,41365900,ABEV3.SA,-0.270000,12.052397,NaN,11.33,NaN,NaN,NaN,0.270000,0.0,14.655374,11.27,14.36,25.320278,-74.679722,-0.169565,-0.012098,-0.096772,-2606670447


In [20]:
test_day_AMBEV['Date'] = pd.to_datetime(test_day_AMBEV['Date'])

# Definir a data inicial para filtrar
inicio = '2024-12-05'  # Data inicial

# Filtrar as linhas onde a data é maior ou igual a 'inicio'
test_day_AMBEV = test_day_AMBEV[test_day_AMBEV['Date'] >= inicio]

test_day_AMBEV.tail(15)


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5720,2024-12-10,14.33,14.50,14.27,14.27,13.524566,22942500,ABEV3.SA,0.060000,13.803438,NaN,14.27,-1.0,12.14,12.675070,0.000000,0.06,78.073184,12.25,14.53,68.133232,-31.866768,0.206047,0.059575,0.099141,-2027668547
5721,2024-12-11,14.27,14.36,13.66,13.80,13.079118,49150600,ABEV3.SA,-0.470000,13.802750,NaN,13.80,-1.0,12.07,12.554056,0.470000,0.00,57.351316,12.25,14.53,68.103080,-31.896920,0.241183,0.095897,0.101865,-2076819147
5722,2024-12-12,13.50,13.58,13.10,13.10,12.415685,49616900,ABEV3.SA,-0.700000,13.662200,NaN,13.10,-1.0,11.74,12.391245,0.700000,0.00,39.386518,12.25,14.53,61.938608,-38.061392,0.254750,0.127667,0.087117,-2126436047
5723,2024-12-13,13.10,13.36,13.01,13.08,12.396729,30867900,ABEV3.SA,-0.020000,13.545760,NaN,13.08,-1.0,11.60,12.232996,0.020000,0.00,38.983936,12.25,14.53,56.831589,-43.168411,0.253188,0.152772,0.066268,-2157303947
5724,2024-12-16,13.08,13.42,12.84,12.84,12.169266,32095500,ABEV3.SA,-0.240000,13.404608,NaN,12.84,-1.0,11.33,12.052397,0.240000,0.00,34.150823,12.25,14.53,50.640711,-49.359289,0.237819,0.169781,0.036456,-2189399447
5725,2024-12-17,13.00,13.19,12.99,13.10,12.415685,39564500,ABEV3.SA,0.260000,13.343687,NaN,13.10,NaN,NaN,NaN,0.000000,0.26,42.986323,12.25,14.53,47.968714,-52.031286,0.218208,0.179466,0.016082,-2228963947
5726,2024-12-18,13.07,13.08,12.75,12.76,12.093445,40864200,ABEV3.SA,-0.340000,13.226949,NaN,12.76,NaN,NaN,NaN,0.340000,0.00,35.748723,12.25,14.53,42.848658,-57.151342,0.191043,0.181782,-0.012177,-2269828147
5727,2024-12-19,12.76,12.91,12.64,12.78,12.112400,38104000,ABEV3.SA,0.020000,13.137559,NaN,12.78,NaN,NaN,NaN,0.000000,0.02,36.474660,12.60,14.53,27.852807,-72.147193,0.160453,0.177516,-0.030867,-2307932147
5728,2024-12-20,12.08,12.30,12.01,12.23,12.230000,86074600,ABEV3.SA,-0.550000,12.956047,NaN,12.23,NaN,NaN,NaN,0.550000,0.00,26.848892,12.01,14.53,37.541562,-62.458438,0.120178,0.166048,-0.053391,-2394006747
5729,2024-12-23,12.11,12.34,11.99,12.22,12.220000,40181700,ABEV3.SA,-0.009999,12.808838,NaN,12.22,NaN,NaN,NaN,0.009999,0.00,26.701074,11.99,14.53,32.237725,-67.762275,0.075511,0.147941,-0.072054,-2434188447


##### Carregar modelo aprendido geral

In [15]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model_D10_0.99.pkl')

#### Carregar modelo aprendido especifico.

In [17]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model_Ambev.pkl')

In [7]:
Features = test_day_AMBEV[['RSI','k_percent','r_percent','Price_Rate_Of_Change','MACD','MACD_EMA','On Balance Volume']]
Features.head()

,RSI,k_percent,r_percent,Price_Rate_Of_Change,MACD,MACD_EMA,On Balance Volume
5717,86.722945,51.583477,-48.416523,0.066920,0.052492,-0.040186,-2122382047
5718,77.627058,57.280530,-42.719470,0.080322,0.106557,-0.010837,-2075932847
5719,77.160149,63.017409,-36.982591,0.089671,0.158135,0.022957,-2050611047
5720,78.073184,68.133232,-31.866768,0.099141,0.206047,0.059575,-2027668547
5721,57.351316,68.103080,-31.896920,0.101865,0.241183,0.095897,-2076819147


In [16]:
Features_array = Features.to_numpy()

# Fazer previsões usando o modelo carregado
predictions = rand_frst_clf_Ambev.predict(Features_array)

# Adicionar as previsões em uma nova coluna chamada 'Prediction'
test_day_AMBEV['Prediction'] = predictions

test_day_AMBEV.tail(30)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5717,2024-12-05,14.05,14.46,14.00,14.42,13.666730,38794000,ABEV3.SA,0.490000,13.389995,-1.0,14.42,-1.0,12.78,13.137559,0.000000,0.49,86.722945,12.25,14.46,51.583477,-48.416523,0.052492,-0.040186,0.066920,-2122382047
5718,2024-12-06,14.30,14.53,14.15,14.22,13.477178,46449200,ABEV3.SA,-0.200000,13.555996,-1.0,14.22,-1.0,12.23,12.956047,0.200000,0.00,77.627058,12.25,14.53,57.280530,-42.719470,0.106557,-0.010837,0.080322,-2075932847
5719,2024-12-09,14.22,14.33,14.14,14.21,13.467700,25321800,ABEV3.SA,-0.010000,13.686797,-1.0,14.21,-1.0,12.22,12.808838,0.010000,0.00,77.160149,12.25,14.53,63.017409,-36.982591,0.158135,0.022957,0.089671,-2050611047
5720,2024-12-10,14.33,14.50,14.27,14.27,13.524566,22942500,ABEV3.SA,0.060000,13.803438,1.0,14.27,-1.0,12.14,12.675070,0.000000,0.06,78.073184,12.25,14.53,68.133232,-31.866768,0.206047,0.059575,0.099141,-2027668547
5721,2024-12-11,14.27,14.36,13.66,13.80,13.079118,49150600,ABEV3.SA,-0.470000,13.802750,-1.0,13.80,-1.0,12.07,12.554056,0.470000,0.00,57.351316,12.25,14.53,68.103080,-31.896920,0.241183,0.095897,0.101865,-2076819147
5722,2024-12-12,13.50,13.58,13.10,13.10,12.415685,49616900,ABEV3.SA,-0.700000,13.662200,1.0,13.10,-1.0,11.74,12.391245,0.700000,0.00,39.386518,12.25,14.53,61.938608,-38.061392,0.254750,0.127667,0.087117,-2126436047
5723,2024-12-13,13.10,13.36,13.01,13.08,12.396729,30867900,ABEV3.SA,-0.020000,13.545760,1.0,13.08,-1.0,11.60,12.232996,0.020000,0.00,38.983936,12.25,14.53,56.831589,-43.168411,0.253188,0.152772,0.066268,-2157303947
5724,2024-12-16,13.08,13.42,12.84,12.84,12.169266,32095500,ABEV3.SA,-0.240000,13.404608,1.0,12.84,-1.0,11.33,12.052397,0.240000,0.00,34.150823,12.25,14.53,50.640711,-49.359289,0.237819,0.169781,0.036456,-2189399447
5725,2024-12-17,13.00,13.19,12.99,13.10,12.415685,39564500,ABEV3.SA,0.260000,13.343687,1.0,13.10,NaN,NaN,NaN,0.000000,0.26,42.986323,12.25,14.53,47.968714,-52.031286,0.218208,0.179466,0.016082,-2228963947
5726,2024-12-18,13.07,13.08,12.75,12.76,12.093445,40864200,ABEV3.SA,-0.340000,13.226949,1.0,12.76,NaN,NaN,NaN,0.340000,0.00,35.748723,12.25,14.53,42.848658,-57.151342,0.191043,0.181782,-0.012177,-2269828147


In [17]:
test_day_AMBEV.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)

In [10]:
# We need to remove all rows that have an NaN value.
print('Antes de tirar as NaN nós tinhamos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))

# Any row that has a `NaN` value will be dropped.
test_day_AMBEV = test_day_AMBEV.dropna()

# Display how much we have left now.
print('Depois da retirada dos NaN temos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))


Antes de tirar as NaN nós tinhamos 16 linhas e 26 colunas
Depois da retirada dos NaN temos 6 linhas e 26 colunas


In [11]:
# Comparar valores e calcular a precisão
test_day_AMBEV['Match'] = (test_day_AMBEV['Prediction'] == test_day_AMBEV['Verify']).astype(int)
accuracy = test_day_AMBEV['Match'].mean()  # Proporção de acertos (iguais / total)

print(f"\nPrecisão do modelo: {accuracy:.2%}")


Precisão do modelo: 83.33%
